# Retrieve Information from Specific Data Corpus

## Set up Azure OpenAI

In [16]:
import os
import openai
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv("credentials.env")

openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") # Api base is the 'Endpoint' which can be found in Azure Portal where Azure OpenAI is created. It looks like https://xxxxxx.openai.azure.com/
openai.api_version ="2023-03-15-preview"
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")

In [1]:
"""from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azureml.core import Workspace

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

try:
    ml_client = MLClient.from_config(credential=credential, path="workspace.json")
except Exception as ex:
    raise Exception(
        "Failed to create MLClient from config file. Please modify and then run the above cell with your AzureML Workspace details."
    ) from ex
    # ml_client = MLClient(
    #     credential=credential,
    #     subscription_id="",
    #     resource_group_name="",
    #     workspace_name=""
    # )

ws = Workspace(
    subscription_id=ml_client.subscription_id,
    resource_group=ml_client.resource_group_name,
    workspace_name=ml_client.workspace_name,
)
print(ml_client)

keyvault = ws.get_default_keyvault()
aoai_endpoint=keyvault.get_secret(name="aoai-endpoint")
aoai_key=keyvault.get_secret(name="key")"""

Found the config file in: workspace.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f3897ba4c40>,
         subscription_id=fe38c376-b42a-4741-9e7c-f5d7c31e5873,
         resource_group_name=yelizkilinc-rg,
         workspace_name=aml-prod)


In [2]:
"""import openai
# Set up Azure OpenAI
openai.api_type = "azure"
openai.api_base = aoai_endpoint # Api base is the 'Endpoint' which can be found in Azure Portal where Azure OpenAI is created. It looks like https://xxxxxx.openai.azure.com/
openai.api_version = "2023-03-15-preview"
openai.api_key = aoai_key"""

## Deploy a language model if it hasn't been deployed before

In [17]:
# list models deployed with embeddings capability
deployment_id = None
result = openai.Deployment.list()
desired_model = 'text-embedding-ada-002'

# check if desired model is already deployed
for deployment in result.data:
    if deployment["status"] != "succeeded":
        continue
    
    if deployment['model'] != desired_model:
        continue
    
    deployment_id = deployment["id"]; print(deployment_id)
    break

# if not model deployed, deploy one
if not deployment_id:
    print('No deployment with status: succeeded found.')
    model = desired_model

    # Now let's create the deployment
    print(f'Creating a new deployment with model: {model}')
    result = openai.Deployment.create(model=model, scale_settings={"scale_type":"standard"})
    deployment_id = result["id"]
    print(f'Successfully created {model} with deployment_id {deployment_id}')
else:
    print(f'Found a succeeded deployment that supports embeddings with id: {deployment_id}.')

text-embedding-ada-002
Found a succeeded deployment that supports embeddings with id: text-embedding-ada-002.


## Load Data

In [18]:
import pandas as pd
fname = 'data/bbc-news-data.csv'
df_orig = pd.read_csv(fname, delimiter='\t', index_col=False)

In [19]:
import numpy as np

DEVELOPMENT = False  # Set this to True for development on small subset of data

if DEVELOPMENT:
    # Sub-sample for development
    df = df_orig.sample(n=20, replace=False, random_state=9).copy() # Set sample size
else:
    df = df_orig.copy()

df

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...
...,...,...,...,...
2220,tech,397.txt,BT program to beat dialler scams,BT is introducing two initiatives to help bea...
2221,tech,398.txt,Spam e-mails tempt net shoppers,Computer users across the world continue to i...
2222,tech,399.txt,Be careful how you code,A new European directive could put software w...
2223,tech,400.txt,US cyber security chief resigns,The man making sure US computer networks are ...


In [20]:
df[df['category']=='entertainment']

,category,filename,title,content
510,entertainment,001.txt,Gallery unveils interactive tree,A Christmas tree that can receive text messag...
511,entertainment,002.txt,Jarre joins fairytale celebration,French musician Jean-Michel Jarre is to perfo...
512,entertainment,003.txt,Musical treatment for Capra film,The classic film It's A Wonderful Life is to ...
513,entertainment,004.txt,Richard and Judy choose top books,The 10 authors shortlisted for a Richard and ...
514,entertainment,005.txt,Poppins musical gets flying start,The stage adaptation of children's film Mary ...
...,...,...,...,...
891,entertainment,382.txt,Last Star Wars 'not for children',The sixth and final Star Wars movie may not b...
892,entertainment,383.txt,French honour for director Parker,British film director Sir Alan Parker has bee...
893,entertainment,384.txt,Robots march to US cinema summit,Animated movie Robots has opened at the top o...
894,entertainment,385.txt,Hobbit picture 'four years away',Lord of the Rings director Peter Jackson has ...


## Unstrcutured data to structured data

In [21]:
def send_message(messages, model_name, max_response_tokens=500):
    response = openai.ChatCompletion.create(
        engine=model_name,
        messages=messages,
        temperature=0.5,
        max_tokens=max_response_tokens,
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response

In [22]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"
print(system_message)

You are a helpful assistant.


## Sample Queries

In [23]:
idx = 513 #index of the selected text

# prompt postifx
prompt_prefix = """ 
  \n\n Extract author and books as columns from the given context in a table format. 
"""
# build prompt
prompt =prompt_prefix + "### Context:"+ df['content'].loc[idx] +"###"

user_message=prompt
print(prompt)

 
  

 Extract author and books as columns from the given context in a table format. 
### Context: The 10 authors shortlisted for a Richard and Judy book award in 2005 are hoping for a boost in sales following the success of this year's winner.  The TV couple's interest in the book world coined the term "the Richard & Judy effect" and created the top two best-selling paperbacks of 2004 so far. The finalists for 2005 include Andrew Taylor's The American Boy and Robbie Williams' autobiography Feel. This year's winner, Alice Sebold's The Lovely Bones, sold over one million. Joseph O'Connor's Star of the Sea came second and saw sales increase by 350%. The best read award, on Richard Madeley and Judy Finnigan's Channel 4 show, is part of the British Book Awards. David Mitchell's Booker-shortlisted novel, Cloud Atlas, makes it into this year's top 10 along with several lesser known works.  "There's no doubt that this year's selection of book club entries is the best yet. If anything, the cho

In [24]:
# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

In [22]:
# messages.clear()

In [25]:
max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)

In [26]:
response['choices'][0]['message']['content']

"| Author         | Book                |\n|----------------|---------------------|\n| Andrew Taylor  | The American Boy    |\n| Robbie Williams| Feel                |\n| Alice Sebold   | The Lovely Bones    |\n| Joseph O'Connor| Star of the Sea     |\n| David Mitchell | Cloud Atlas         |"

In [27]:
idx = 891 #index of the selected text

# prompt postifx
prompt_postfix = """ 
  \n\n Extract Star Wars movie series and associated ratings from the text above into a table. Every row in the table should start in a new line. """
# build prompt
prompt = df['content'].loc[idx] + prompt_postfix; print(prompt)


 The sixth and final Star Wars movie may not be suitable for young children, film-maker George Lucas has said.  He told US TV show 60 Minutes that Revenge of the Sith would be the darkest and most violent of the series. "I don't think I would take a five or six-year-old to this," he told the CBS programme, to be aired on Sunday. Lucas predicted the film would get a US rating advising parents some scenes may be unsuitable for under-13s. It opens in the UK and US on 19 May. He said he expected the film would be classified PG-13 - roughly equivalent to a British 12A rating.  The five previous Star Wars films have all carried less restrictive PG - parental guidance - ratings in the US. In the UK, they have all been passed U - suitable for all - with the exception of Attack of The Clones, which got a PG rating in 2002. Revenge of the Sith - the third prequel to the original 1977 Star Wars film - chronicles the transformation of the heroic Anakin Skywalker into the evil Darth Vader as he tra

In [28]:
user_message=prompt

messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

# query
max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)

In [29]:
response['choices'][0]['message']['content']

'| Movie Title | Release Year | US Rating | UK Rating |\n| --- | --- | --- | --- |\n| Star Wars (Original) | 1977 | PG | U |\n| Attack of The Clones | 2002 | PG | PG |\n| Revenge of the Sith | 2005 | Predicted PG-13 | Equivalent to 12A |'